# Analysis: Descriptions' and Annotations' Lengths
## Post Annotation and Aggregation

Outputs the files:
  * `../data/analysis_data/descriptions_with_counts.csv`: adds columns to `descriptions.csv` for word counts and sentence counts, where words are alphanumeric tokens (punctuation excluded)
  * `../data/analysis_data/descs_stats.csv`: contains the count, minimum, maximum, average, and standard deviation of all descriptions and each type of description
  * `../data/crc_metadata/all_descs_with_offsets.csv`: contains one row for every description in the annotated datasets with columns for the descriptions' corresponding id, eadid, file, start offset, and end offset

***

**Table of Contents**

[0.](#0) Loading

[1.](#1) Lengths of Descriptions and Annotations

  * [Lengths of Descriptions](#1.1)
  
  * TO DO: [Lengths of Annotations](#1.2)
  
[2.](#2) Offsets of Descriptions

***

<a id="0"></a>
### 0. Loading
First, begin by loading Python programming libraries and the dataset to be analyzed.

In [2]:
import utils  # import custom functions

import pandas as pd
import numpy as np
import string, csv, re, os, sys #,json

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
# nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.tag import pos_tag
from nltk.text import Text
from nltk.probability import FreqDist
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt

from . import _distributor_init  # for mkl-service (Intel(r) MKL initialization)

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "/Users/lucy/Desktop/repos/gender-bias/env/bin/python"
  * The NumPy version is: "1.23.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/lucy/Desktop/repos/gender-bias/env/lib/python3.9/site-packages/numpy/core/_multiarray_umath.cpython-39-darwin.so, 0x0002): Library not loaded: '@rpath/libmkl_rt.1.dylib'
  Referenced from: '/Users/lucy/Desktop/repos/gender-bias/env/lib/python3.9/site-packag

In [34]:
# dir_path = "../data/aggregated_data/"
# data_files = ["aggregated_final.csv", "aggregated_with_annotator_eadid_note_cols.csv", 
#               "aggregated_with_eadid_descid_desc_cols.csv"]

In [35]:
# df = pd.read_csv(dir_path+data_files[2], index_col=0)
# df.head()

In [36]:
# print("Rows:",df.shape[0], "\nColumns:",df.shape[1])  # Rows: 55260, Columns: 10

<a id="1"></a>
## 1. Lengths of Descriptions and Annotations
**Find the minimum, maximum, average, and standard deviation of word and sentence counts...**
* Per description (by `desc_id` - a.k.a. per "document" for document classifiers)
* Per metadata field (Title, Biographical / Historical, Scope and Contents, and Processing Information)
* Per collection (identifiable with the `eadid` column)
* Per annotation label (Omission, Stereotype, Generalization, etc.)
* Per annotation category (Person Name, Linguistic, Contextual)

<a id="1.1"></a>
### 1.1 Lengths of Descriptions

In [37]:
descs_path = "../data/crc_metadata/all_descriptions.csv"     # descriptions in column of CSV file

In [38]:
desc_df = pd.read_csv(descs_path, index_col=0)
desc_df.head()

,eadid,description,field,desc_id
0,AA5,Professor James Aitken White was a leading Sco...,Biographical / Historical,0
1,AA5,Papers of The Very Rev Prof James Whyte (1920-...,Title,1
2,AA6,Rev Thomas Allan was born on 16 August 1916 in...,Biographical / Historical,2
3,AA6,Papers of Rev Tom Allan (1916-1965)\n\n,Title,3
4,AA7,Alec Cheyne was born on 1 June 1924 in Errol i...,Biographical / Historical,4


In [39]:
# # Remove metadata field name from each description
# new_descs = []
# descs = list(desc_df.description)
# fields = list(desc_df.field)
# i = 0
# maxI = len(descs)
# while i < maxI:
#     d, f = descs[i], fields[i]
#     to_remove = f+":\n"
#     d = d.replace(to_remove,"")
#     new_descs += [d]
#     i += 1
# assert len(new_descs) == len(descs)
# # new_descs[:10]            # Looks good

In [40]:
# # Update the CSV file
# desc_df.description = new_descs
# desc_df.head()
# desc_df.to_csv(descs_path)

In [42]:
# Write each description to a txt file named with desc_id
ids = list(desc_df.desc_id)
zero_padding = len(str(ids[-1]))
desc_txt_dir = "data/descriptions/"
i, maxI = 0, len(ids)
while i < maxI:
    d_id = str(ids[i])
    padding = zero_padding - len(d_id)  # pad with zeros so file order aligns with DataFrame order
    id_str = ("0"*padding) + d_id
    filename = "description"+id_str+".txt"
    f = open((desc_txt_dir+filename), "w", encoding="utf8")
    f.write(new_descs[i])
    f.close()
    i += 1
print("Files written to "+desc_txt_dir)

In [ ]:
corpus = PlaintextCorpusReader("data/descriptions/", "description\d+.txt", encoding="utf8")
# print(len(corpus.fileids()), desc_df.shape[0])  # Looks good
print(corpus.fileids()[-20:]) # Looks good

#### Length per Description

In [ ]:
desc_words, desc_lower_words, desc_sents = utils.getWordsSents(corpus)
print(desc_words[0][:10])
print(desc_lower_words[0][:10])
print(desc_sents[0][:2])

In [ ]:
# Add word and sentence counts to DataFrame/CSV of descriptions
word_count = [len(word_list) for word_list in desc_words]  # includes digits but not punctuation
sent_count = [len(sent_list) for sent_list in desc_sents]
print(word_count[:2], sent_count[:4])  # Looks good
# len(desc_sents[2]) # 14

In [ ]:
desc_df.insert(len(desc_df.columns), "word_count", word_count)
desc_df.insert(len(desc_df.columns), "sent_count", sent_count)
desc_df.head()

In [ ]:
desc_df.to_csv("descriptions_with_counts.csv")  # write a new CSV file with the word and sentence counts

In [ ]:
desc_df = desc_df.reset_index()
desc_df.head(1)

In [ ]:
desc_df_stats = utils.makeDescribeDf("All", desc_df)
desc_df_stats

#### Lengths per Metadata Field

In [ ]:
field = "Biographical / Historical"
bh_stats = utils.makeDescribeDf(field, desc_df)
bh_stats

In [ ]:
field = "Scope and Contents"
sc_stats = utils.makeDescribeDf(field, desc_df)
sc_stats

In [ ]:
field = "Processing Information"
pi_stats = utils.makeDescribeDf(field, desc_df)
pi_stats

In [ ]:
field = "Title"
t_stats = utils.makeDescribeDf(field, desc_df)
t_stats

#### Combine the Statistics

In [ ]:
df_stats = pd.concat([desc_df_stats, t_stats, sc_stats, bh_stats, pi_stats], axis=0)
df_stats

In [ ]:
df_stats.to_csv("../data/analysis_data/descs_stats.csv")

#### Prepare data for visualization in Observable

In [ ]:
df_descs = pd.read_csv("../data/analysis_data/descriptions_with_counts.csv", index_col=0)
df_descs.head()

<a id="1.2"></a>
### 1.2 Length of Annotations

* Dataset: `annot-post/data/aggregated_final.csv`

<a id="2"></a>
### 2. Offsets of Descriptions

**Get the start and end offset of every description so that automated labels can be exported as .ann files for visualization with brat.**

The [standoff format](https://brat.nlplab.org/standoff.html) that the brat rapid annotation tool uses records the start offset and end offset of annotated text spans where:
* The **start offset** is the index of the *first character* in the annotated text span (which is also the number of characters in the document preceding the beginning of the annotated text span)
* The **end offset** is the index of the character *after the annotated text span* (which means the end offset corresponds to the character immediately following the annotated text span)

This means that the start offset of the first description of each document will be 0 and the end offset of the last description of each document will equal the length (number of characters) of the document.  There are multiple descriptions for each document, so we need to determinen the intermediate start and end offsets as well, which we'll add as a column to the file `../data/crc_metadata/all_descriptions.csv`.

## TO DO: RE-ASSIGN DESC IDS TO ANNOTATION DATA

In [2]:
doc_path = "../data/crc_metadata/descriptions_brat/"
file_type = ".txt"  # Read in only the PlainText files

In [3]:
filenames = os.listdir(doc_path)
filenames = [f for f in filenames if f[-4:] == file_type] # the descriptions are in the txt files
print(filenames[:3])

['Coll-227_00100.txt', 'La_03600.txt', 'PJM_03000.txt']


In [4]:
metadata_field_names = ["Title", "Scope and Contents", "Biographical / Historical", "Processing Information"]
            
# INPUT: file path to a document of metadata descriptions (str)
# OUTPUT: a dictionary of metadata description ids and the associated 
#         description text, field name and offsets contained in the input file
def getDescriptionsInFiles(dirpath, file_list, fieldnames=metadata_field_names):
    desc_dict = dict()
    did = 0
    for filename in file_list:

        # Get a string of the input file's text (metadata descriptions)
        f_string = open(os.path.join(dirpath+filename),'r').read()
        
        for fieldname in fieldnames:
            pattern = "(?<={}:\n).+".format(fieldname)
            match_list = re.findall(pattern, f_string)
            if len(match_list) > 0:
                for match in match_list:
                    desc_dict[did] = dict.fromkeys(["description", "field", "file", "start_offset", "end_offset"])
                    desc_dict[did]["description"] = match
                    desc_dict[did]["field"] = fieldname
                    desc_dict[did]["file"] = filename
                    desc_dict[did]["start_offset"] = f_string.find(match)
                    desc_dict[did]["end_offset"] = f_string.find(match) + len(match) + 1
                    did += 1
                    
    return desc_dict

In [5]:
descs_details = getDescriptionsInFiles(doc_path, filenames)

Great!  Now create a DataFrame of the description data:

In [27]:
ids_col = list(descs_details.keys())
desc_col, field_col, file_col, eadid_col, start_offset_col, end_offset_col = [], [], [], [], [], []
for desc_id in ids_col:
    desc_dict = descs_details[desc_id]
    
    eadid = (re.findall("^.*(?=_\d+.txt)", desc_dict["file"]))[0]
    eadid_col += [eadid]
    
    field_col += [desc_dict["field"]]
    
    file_col += [desc_dict["file"]]
    
    desc_col += [desc_dict["description"]]
    
    start_offset_col += [desc_dict["start_offset"]]
    end_offset_col += [desc_dict["end_offset"]]

new_descs_df = pd.DataFrame({
    "desc_id":ids_col, "eadid":eadid_col, "field":field_col, "file":file_col, 
    "description":desc_col, "desc_start_offset":start_offset_col, "desc_end_offset":end_offset_col
})

new_descs_df.head()

,desc_id,eadid,field,file,description,desc_start_offset,desc_end_offset
0,0,Coll-227,Title,Coll-227_00100.txt,Records of the Phrenological Society of Edinburgh,29,79
1,1,Coll-227,Scope and Contents,Coll-227_00100.txt,The records of the Phrenological Society inclu...,100,610
2,2,Coll-227,Biographical / Historical,Coll-227_00100.txt,The Phrenological Society of Edinburgh was for...,638,2277
3,3,La,Title,La_03600.txt,"Letter: 1825 Jan. 10, 27 Lower Belgrave Place ...",7,117
4,4,La,Title,La_03600.txt,"Letter: 1825 Mar. 1, 27 Lower Belgrave Place [...",125,223


Write the data to a CSV file:

In [10]:
new_descs_df.to_csv("../data/crc_metadata/descs_with_offsets.csv")

Now assign description IDs from this DataFrame to the aggregated annotated datasets:

In [24]:
dir_path = "../data/aggregated_data/"
df_grouped = pd.read_csv(dir_path+"desc_field_descid_label_eadid.csv", index_col=0)
df_agg = pd.read_csv(dir_path+"aggregated_with_eadid_descid_desc_cols.csv", index_col=0)

In [25]:
df_grouped.head()

,description,field,desc_id,label,eadid
8054,Professor James Aitken White was a leading Sco...,Biographical / Historical,0,"{'Occupation', 'Stereotype', 'Masculine', 'Gen...",AA5
5560,Papers of The Very Rev Prof James Whyte (1920-...,Title,1,"{'Stereotype', 'Masculine', 'Unknown'}",AA5
8321,Rev Thomas Allan was born on 16 August 1916 in...,Biographical / Historical,2,"{'Gendered-Pronoun', 'Stereotype', 'Unknown', ...",AA6
5550,Papers of Rev Tom Allan (1916-1965)\n\n,Title,3,"{'Masculine', 'Unknown'}",AA6
633,Alec Cheyne was born on 1 June 1924 in Errol i...,Biographical / Historical,4,"{'Gendered-Pronoun', 'Stereotype', 'Unknown', ...",AA7


In [34]:
# indeces1 = ["description", "field", "eadid"]
# new_descs_subdf = new_descs_df.drop(columns=["file","desc_start_offset","desc_end_offset"])
# df_grouped = df_grouped.drop(columns=["desc_id"])
df_grouped_joined = df_grouped.set_index(["description","field","eadid"]).join(new_descs_df.set_index(["description","field","eadid"]), how="outer", lsuffix="_old", rsuffix="_new")

In [35]:
df_grouped_joined.head()

,,,desc_id_old,label,desc_id_new,file,desc_start_offset,desc_end_offset
description,field,eadid,,,,,,
""" ""He is an immature looking boy who has had a good deal of physical illness during the last few years. His home conditions are not favourable in so far as his relatives are over-anxious and over-protective. He never mixed with other children while he was small and developed an anxiety neurosis as soon as he started school. At present it seems advisable for him to go to school mornings only and our Social Worker will get in touch with the teacher to discuss the problem of improving his adjustment at school. He will also need to have psychotherapy for a considerable period and I have arranged for him to come to the Clinic once a week."" Had aptosis which the doctor could not account for – encephalitis? – now all right. His mother was 15 or 16 when he was born and his father was the same age. A peculiar family, all a little queer. They were entertainers.""",Scope and Contents,EUA_IN1,NaN,NaN,41051.0,EUA_IN1_46900.txt,2654.0,3519.0
""" ""No control"", was definitely silly, attended Special Class for a time. Nomadic family, have now moved? Should be in an institution? Got into trouble for window breaking, a hopeless looking child. Illegitimate.""Relatives in survey: Mother 7.52; sibling 7.14",Scope and Contents,EUA_IN1,NaN,NaN,31584.0,EUA_IN1_47600.txt,3077.0,3336.0
""" ""No just thorough, light-headed"". Has been on a charge for stealing a bicycle which he destroyed; also suspected of breaking shop windows but this was never proved. Sits for hours in front of the fire with his head in his hands. Solitary, no mates. Now working in the mill. Father, decent. Mother dead. Siblings all OK. Steady work record.""",Scope and Contents,EUA_IN1,NaN,NaN,34014.0,EUA_IN1_47200.txt,9976.0,10319.0
""" ""This appears to be a reaction to her mother's fostering and mollycoddling. The mother appears to be convinced that her daughter is physcially and nervously much below par and even during a short interview it was obvious that she constantly strengthens these tendencies in her daughtr, so much so that the girl herself is now convinced that she is unable to stand up to everyday life"". Dr confirms - spoilt by mother, complains of tiredness and backache. PC - very decent family, give no trouble.""",Scope and Contents,EUA_IN1,NaN,NaN,63099.0,EUA_IN1_41900.txt,7301.0,7801.0
"""""A big fat dirty lump"" - Dr. Nurse says she is decent but never gets out of the bit. Dr says she is ""haughly"".""Husband 18.20, son 18.22",Scope and Contents,EUA_IN1,NaN,NaN,30479.0,EUA_IN1_38200.txt,3666.0,3803.0
